# ComeBackHome with provide current posistion

In [3]:
from numba import jit, njit
import random
import numpy as np

In [4]:
@njit 
def getAgentSize():
    return 10
@njit
def getStateSize():
    return 5
@njit   
def getActionSize():
    return 4

In [5]:
@njit 
def getReward(p_state):
    if p_state[0] == p_state[1] and p_state[2] == p_state[3]:
        return 1
    else:
        return -1
@njit 
def checkEnded(e_state):
    size = 10
    for i in range(0,10):
        if  e_state[size*9*size+i*2] == e_state[size*9*size+20] and e_state[size*9*size+i*2+1] == e_state[size*9*size+21] :
            return i
    return -1
    

In [6]:
@njit   
def initEnv():
    n = 1
    p = 0.5
    # the size of the matrix 
    size = 10
    grid = np.random.binomial(n,p, size=(size,size))
    first_row = grid[0]
    first_row[first_row == 1] = 0
    grid[0] = first_row
    for i in range(1,size):
        grid[i,size-1] = 1
    # create environment
    # from 0 to 899: map 
    # from 900 to 919: player position
    # from 920 to 921: end position
    # from 922 to 923: end position
    # 924: turn

    env = np.full(size*9*size+23,0)

    # create the map 
    map = np.full((size*3, size*3),1)
    i = 0
    j = 0
    while i < size:
        w = i*3 + 1
        while j < size:
            k = j*3 + 1
            toss = grid[i,j]
            map[w,k] = 0 # ' '
            if toss == 0 and k+2 < size*3:
                map[w,k+1] = 0 # ' '
                map[w,k+2] = 0 #' '
            if toss == 1 and w-2 >=0:
                map[w-1,k] = 0 #' '
                map[w-2,k] = 0 #' '
            j = j + 1
        i = i + 1
        j = 0

    # create end and start point
    rows, cols = (np.where(map == 0))


    # start point
    start = random.randint(0, len(rows)-1)
    print('choice start',rows[start],cols[start])
    
    # end point 
    end = random.randint(0, len(rows)-1)
    if end == start :
        end = random.randint(0, len(rows)-1)
    print('choice end ',rows[end],cols[end])

    
    map = map.reshape(1,size*9*size)[0]
    # print(map)
    env[np.where(map== 1)[0]] = 1
    env[np.where(map== 0)[0]] = 0

    # current_agent_posistion_player
    for i in range(0,10):
        env[size*9*size+i*2]= rows[start]
        env[size*9*size+i*2+1]= cols[start]

    # end position
    env[size*9*size+20] = rows[end]
    env[size*9*size+21] = cols[end]

    # Turn
    env[size*9*size+22] = 0

    return env

@njit 
def getAgentState(e_state):
    size = 10
    pIdx = e_state[size*9*size+22] % 10 # player index

    p_state = np.full(5,0) 

    # The position of the agent
    p_state[0] = e_state[size*9*size+pIdx*2] 
    p_state[1] = e_state[size*9*size+pIdx*2+1]

    # The position of the end
    p_state[2] = e_state[size*9*size+20]
    p_state[3] = e_state[size*9*size+21] 

    p_state[4] = pIdx # player position

    return p_state

@njit
def stepEnv(action,env):
    size = 10
    #player index
    pIdx = env[size*9*size+22] % 10
    # current position
    cur_x = env[size*9*size+pIdx*2]
    cur_y = env[size*9*size+pIdx*2+1]

    # go left
    if action == 0:
        if cur_y - 1 >= 0 and env[(cur_x)*size*size+cur_y-1] == 0:
            env[size*9*size+pIdx*2] = cur_x 
            env[size*9*size+pIdx*2+1] = cur_y-1

    # go straight
    if action == 1:
        if cur_x - 1 >= 0 and env[(cur_x-1)*size*size+cur_y] == 0:
            env[size*9*size+pIdx*2] = cur_x-1
            env[size*9*size+pIdx*2+1] = cur_y 

    # go down
    if action == 2:
        if cur_x + 1 < size*3 and env[(1+cur_x)*size*size+cur_y] == 0:
            env[size*9*size+pIdx*2] = cur_x+1
            env[size*9*size+pIdx*2+1] = cur_y 

    # go right
    if action == 3:
        if cur_y + 1 < size*3 and env[(cur_x)*size*size+cur_y+1] == 0:
            env[size*9*size+pIdx*2] = cur_x 
            env[size*9*size+pIdx*2+1] = cur_y+1

    env[size*9*size+22]+=1
@njit 
def getValidActions(p_state):
    validActions = np.full(4, 1)
    return validActions

In [7]:
def player_random(p_state, temp_file, per_file2):
    arr_action = getValidActions(p_state)
    arr_action = np.where(arr_action == 1)[0]
    act_idx = np.random.randint(0, len(arr_action))
    return arr_action[act_idx], temp_file, per_file2


from os import environ
import numpy as np
from numba.typed import List

def run(listAgent, perData):
    size = 10
    env = initEnv()
    tempData = []
    for _ in range(10):
        dataOnePlayer = List()
        dataOnePlayer.append(np.array([[0.]]))
        tempData.append(dataOnePlayer)

    winner = -1
    print('Start State')
    for i in range(0,10):
        print(f'P{i+1}:',end=" ")
        print(env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
    print(' ')
    print(' ')
    
    output = env[0:size*9*size].reshape(size*3,size*3)

    print(output)

    output_grid1 = np.empty([size*3, size*3],dtype=str)
    output_grid1[:] = '#'
    for i in range(len(output)):
        for j in range(len(output)):
            if output[i,j] == 0:
                output_grid1[i,j] = ' '
    for elm in output_grid1:
        print(" ".join(elm))
    
    while env[size*9*size+22]<4000:
        pIdx = env[size*9*size+22]  % 10
        # for i in range(0,10):
        #     print(f'P{i+1}:',end=" ")
        #     print(env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
        # print(' ')
        # print(f'Turn {env[size*9*size+22]+1}',end=" ")
        action, tempData[pIdx], perData = listAgent[pIdx](getAgentState(env), tempData[pIdx], perData)
        if action == 0:  #go straight   
            print(f'Player: {pIdx+1} đi trái')
        if action == 1 : # go left :
            print(f'Player: {pIdx+1}  đi thẳng')
        if action == 2 : # go right :
            print(f'Player: {pIdx+1}  đi lùi')
        if action == 3 : # go back :
            print(f'Player: {pIdx+1}  đi phải')
        stepEnv(action, env)
        winner = checkEnded(env)
        
        if winner != -1:
            for i in range(0,10):
                print(f'P{i+1}:',end=" ")
                print('X',env[size*9*size+i*2],env[size*9*size+i*2+1],end=" ")
            print(' ')
            print('Player winner : Player ',winner+1)
            print('số turn',env[size*9*size+22]+1)
            break
    
    for pIdx in range(10):
        env[size*9*size+22] = pIdx
        action, tempData[pIdx], perData = listAgent[pIdx](getAgentState(env), tempData[pIdx], perData)
    
    return winner, perData


def main(listAgent, times, perData, printMode=False, k=100):
    if len(listAgent) != 10:
        raise Exception('Hệ thống chỉ cho phép có đúng 10 người chơi!!!')
    
    numWin = np.full(11, 0)
    pIdOrder = np.arange(10)
    for _ in range(times):
        if printMode and _ != 0 and _ % k == 0:
            print(_, numWin)

        np.random.shuffle(pIdOrder)
        shuffledListAgent = [listAgent[i] for i in pIdOrder]
        winner, perData = run(shuffledListAgent, perData)
        # print('w',winner)
        print('real player ',pIdOrder[winner])

        if winner == -1:
            numWin[10] += 1
        else:
            numWin[pIdOrder[winner]] += 1

    if printMode:
        print(_+1, numWin)

    return numWin, perData
list_player = [player_random,player_random,player_random,player_random,player_random,player_random,player_random,player_random,player_random,player_random]
per = []
count,per1 = main(list_player,1,per)

choice start 20 28
choice end  22 14
Start State
P1: 20 28 P2: 20 28 P3: 20 28 P4: 20 28 P5: 20 28 P6: 20 28 P7: 20 28 P8: 20 28 P9: 20 28 P10: 20 28  
 
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1]
 [1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1]
 [1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1]
 [1 0 0 0 0 0 0 0 1 1 0 1 1

# Numba run 

In [8]:
from numba.typed import List

@njit
def numbaRun(p0,p1, p2, p3,p4,p5,p6,p7,p8,p9, perData, pIdOrder):
    env= initEnv()
    tempData = []
    for _ in range(10):
        dataOnePlayer = List()
        dataOnePlayer.append(np.array([[0.]]))
        tempData.append(dataOnePlayer)
    size = 3
    winner = -1
    while env[size*9*size+22] < 400:
        pIdx = env[size*9*size+22] % 10
        if pIdOrder[pIdx] == 0:
            action, tempData[pIdx], perData = p0(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 1:
            action, tempData[pIdx], perData = p1(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 2:
            action, tempData[pIdx], perData = p2(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 3:
            action, tempData[pIdx], perData = p3(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 4:
            action, tempData[pIdx], perData = p4(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 5:
            action, tempData[pIdx], perData = p5(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 6:
            action, tempData[pIdx], perData = p6(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 7:
            action, tempData[pIdx], perData = p7(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 8:
            action, tempData[pIdx], perData = p8(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 9:
            action, tempData[pIdx], perData = p9(getAgentState(env), tempData[pIdx], perData)

        stepEnv(action, env)
        winner = checkEnded(env)
        if winner != -1:
            break
    
    for pIdx in range(10):
        env[52] = pIdx
        if pIdOrder[pIdx] == 0:
            action, tempData[pIdx], perData = p0(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 1:
            action, tempData[pIdx], perData = p1(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 2:
            action, tempData[pIdx], perData = p2(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 3:
            action, tempData[pIdx], perData = p3(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 4:
            action, tempData[pIdx], perData = p4(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 5:
            action, tempData[pIdx], perData = p5(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 6:
            action, tempData[pIdx], perData = p6(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 7:
            action, tempData[pIdx], perData = p7(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 8:
            action, tempData[pIdx], perData = p8(getAgentState(env), tempData[pIdx], perData)
        elif pIdOrder[pIdx] == 9:
            action, tempData[pIdx], perData = p9(getAgentState(env), tempData[pIdx], perData)
            
    return winner, perData

@njit
def numbaMain(p0,p1, p2, p3,p4,p5,p6,p7,p8,p9, times, perData, printMode=False, k=100):
    numWin = np.full(5, 0)
    pIdOrder = np.arange(4)
    for _ in range(times):
        if printMode and _ != 0 and _ % k == 0:
            print(_, numWin)

        np.random.shuffle(pIdOrder)
        winner, perData = numbaRun(p0,p1, p2, p3,p4,p5,p6,p7,p8,p9, perData, pIdOrder)

        if winner == -1:
            numWin[4] += 1
        else:
            numWin[pIdOrder[winner]] += 1
    
    if printMode:
        print(_+1, numWin)

    return numWin, perData
@njit
def numbaRandomBot(state, tempData, perData):
    validActions = getValidActions(state)
    print(state)
    validActions = np.where(validActions==1)[0]
    idx = np.random.randint(0, len(validActions))
    return validActions[idx], tempData, perData
perx = [np.array([np.random.rand(getActionSize()) for _ in range(100)]),np.zeros((1,100)),np.zeros((1,100)),np.zeros((1,100))]
count,per1 = numbaMain(numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,numbaRandomBot,1,perx)
print(count)

In [ ]:
n = 1
p = 0.5
size = 3
grid = np.random.binomial(n,p, size=(size,size))
first_row = grid[0]
first_row[first_row == 1] = 0
grid[0] = first_row
for i in range(1,size):
  grid[i,size-1] = 1

def carve_maze(grid, size):
  output_grid = np.full((size*3, size*3),1)
  i = 0
  j = 0

  while i < size:
    w = i*3 + 1
    while j < size:
      k = j*3 + 1
      toss = grid[i,j]
      output_grid[w,k] = 0 # ' '
      if toss == 0 and k+2 < size*3:
        output_grid[w,k+1] = 0 #' '
        output_grid[w,k+2] = 0 #' '
      if toss == 1 and w-2 >=0:
        output_grid[w-1,k] = 0 #' '
        output_grid[w-2,k] = 0 #' '
      j = j + 1
    i = i + 1
    j = 0
  return output_grid
output = carve_maze(grid, size)
print(output)
for i in range(len(output)):
  if i == 1:
    print('#', end='')
for i in range(len(np.where(output == 0)[0])):
  rows, cols = (np.where(output == 0))
  print(rows[i],cols[i])
# random range
start = random.randint(0, len(rows)-1)
print('choice start',rows[start],cols[start])
end = random.randint(0, len(rows)-1)
print('choice end ',rows[end],cols[end])

[[1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 1]
 [1 0 1 1 0 1 1 0 1]
 [1 0 1 1 0 1 1 0 1]
 [1 0 1 1 0 1 1 0 1]
 [1 1 1 1 0 1 1 0 1]
 [1 1 1 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 0 1]
 [1 1 1 1 1 1 1 1 1]]
#1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 1
2 4
2 7
3 1
3 4
3 7
4 1
4 4
4 7
5 4
5 7
6 4
6 7
7 1
7 2
7 3
7 4
7 7
choice start 1 3
choice end  7 4


# Visualize bàn chơi

In [ ]:
n = 1
p = 0.5
size = 3
grid = np.random.binomial(n,p, size=(size,size))
first_row = grid[0]
first_row[first_row == 1] = 0
grid[0] = first_row
for i in range(1,size):
  grid[i,size-1] = 1

def carve_maze(grid, size):
  output_grid = np.empty([size*3, size*3],dtype=str)
  output_grid[:] = '#'
  # for elm in output_grid:
  #   print(" ".join(elm))
  # print('')
  i = 0
  j = 0

  while i < size:
    w = i*3 + 1
    while j < size:
      k = j*3 + 1
      toss = grid[i,j]
      output_grid[w,k] = ' '
      if toss == 0 and k+2 < size*3:
        output_grid[w,k+1] = ' '
        output_grid[w,k+2] = ' '
      if toss == 1 and w-2 >=0:
        output_grid[w-1,k] = ' '
        output_grid[w-2,k] = ' '
      j = j + 1
    i = i + 1
    j = 0
  return output_grid
output = carve_maze(grid, size)
for elm in output:
  print(" ".join(elm))

In [ ]:
n = 1
p = 0.5
size = 3
grid = np.random.binomial(n,p, size=(size,size))
first_row = grid[0]
first_row[first_row == 1] = 0
grid[0] = first_row
for i in range(1,size):
  grid[i,size-1] = 1

def carve_maze(grid, size):
  output_grid = np.full((size*3, size*3),1)

  i = 0
  j = 0

  while i < size:
    w = i*3 + 1
    while j < size:
      k = j*3 + 1
      toss = grid[i,j]
      output_grid[w,k] = 0
      if toss == 0 and k+2 < size*3:
        output_grid[w,k+1] = 0
        output_grid[w,k+2] = 0
      if toss == 1 and w-2 >=0:
        output_grid[w-1,k] = 0
        output_grid[w-2,k] = 0
      j = j + 1
    i = i + 1
    j = 0
  return output_grid
output = carve_maze(grid, size)
output_grid1 = np.empty([size*3, size*3],dtype=str)
output_grid1[:] = '#'
for i in range(len(output)):
  for j in range(len(output)):
    if output[i,j] == 0:
      output_grid1[i,j] = ' '
for elm in output_grid1:
  print(" ".join(elm))
print('')

# # # # # # # # #
#               #
# # # # # # #   #
# # # # # # #   #
#               #
#   # #   # #   #
#   # #   # #   #
#   # #   # #   #
# # # # # # # # #



In [ ]:
output_grid1 = np.empty([size*3, size*3],dtype=str)
output_grid1[:] = '#'
for i in range(len(output)):
  for j in range(len(output)):
    if output[i,j] == 0:
      output_grid1[i,j] = ' '
print(output_grid1)

[['#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' '#' '#' '#' ' ' '#']
 ['#' '#' '#' '#' '#' '#' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' ' ' '#' '#' ' ' '#' '#' ' ' '#']
 ['#' ' ' '#' '#' ' ' '#' '#' ' ' '#']
 ['#' ' ' '#' '#' ' ' '#' '#' ' ' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#']]
